**Анализ базы данных сервис для чтения книг по подписке**

**Описание исследования**

На волне увеличившегося спроса на чтение книг из-за коронавирусных ограничений наша компания купила крупный сервис для чтения книг по подписке

**Заказчик**

Отдел маркетинга

**Цель исследования**

Проанализировать базу данных купленного сервиса, чтобы определить ценностное предложение для потенциальных пользователей


**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


![Alt Text](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)


Подключимся к базе данных, чтобы создавать к ней запросы для аналитики:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

На примере 5 первых строк каждой таблицы ознакомимся с данными:

In [3]:
# список всех таблиц
table_name = ['books', 'authors', 'publishers', 'ratings', 'reviews'] 

for tn in table_name:
    ratings_query = '''
    SELECT * 
    FROM {}
    LIMIT 5
    '''.format(tn)

    df = pd.io.sql.read_sql(sql=text(ratings_query), con=con)
    print(f"Table: {tn}")
    display(df)

Table: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Table: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Table: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Table: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Table: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
books_count = '''
SELECT COUNT(title)
FROM books 
WHERE publication_date >= '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(books_count), con = con)

,count
0,821


После 1 января 2020 года выпущено 819 книг и брошюр. Есть из чего выбирать.

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [5]:
est = '''
SELECT  b.book_id, b.title, COUNT(DISTINCT(rw.review_id)) as reviews_number, ROUND(AVG(r.rating),1) as avg_rating 
FROM books AS b 
LEFT JOIN ratings AS r ON b.book_id = r.book_id
LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
GROUP BY b.book_id, b.title
ORDER BY reviews_number DESC
'''

pd.io.sql.read_sql(sql=text(est), con = con)

,book_id,title,reviews_number,avg_rating
0,948,Twilight (Twilight #1),7,3.7
1,963,Water for Elephants,6,4.0
2,734,The Glass Castle,6,4.2
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,695,The Curious Incident of the Dog in the Night-Time,6,4.1
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7
996,808,The Natural Way to Draw,0,3.0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,221,Essential Tales and Poems,0,4.0


В каталоге 1000 книг, по которым не так и много обзоров: от 0 до 7. Средние рейтинги разнятся, но можно в будущем отфильтровать только определенный уровень рейтинга.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [6]:
publisher = '''
WITH p AS (
SELECT publisher_id, COUNT(book_id) AS book_number
FROM books
WHERE num_pages > 50
GROUP BY publisher_id 
ORDER BY book_number DESC
LIMIT 1)

SELECT *
FROM publishers
WHERE publisher_id IN (SELECT publisher_id FROM p)
'''

pd.io.sql.read_sql(sql=text(publisher), con = con)

,publisher_id,publisher
0,212,Penguin Books


Самое популярное издательство: Penguin books. Можно следить за новинками этого издательства в первую очередь.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [7]:
est = '''

--отбираем книги с 50 и более оценок--

WITH z1 AS (SELECT book_id, AVG(rating)
FROM ratings AS r
GROUP BY book_id
HAVING COUNT(rating_id) >= 50),

--отбираем авторов и книги с 50 и более оценок--

z2 AS (SELECT *
FROM books
WHERE book_id IN (SELECT book_id FROM z1)),

--присоединяем рейтинги по книгам--
z3 AS (SELECT *
FROM z2
LEFT JOIN z1 ON z2.book_id=z1.book_id),

--вычисляем автора--
z4 AS (SELECT author_id, AVG(avg) AS book_avg_rating
FROM z3
GROUP BY author_id
ORDER BY book_avg_rating DESC
LIMIT 1)

--вычисляем имя автора
SELECT a.author, ROUND(z4.book_avg_rating,2)
FROM authors AS a JOIN z4 ON a.author_id=z4.author_id



'''

pd.io.sql.read_sql(sql=text(est), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.28


Автор с книгами с самыми высокими рейтингами (при достаточно большом количестве оценок у каждой книги) - Джоан Роулинг. Можно подумать о продвижении в сервисе её книг.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [8]:
reviews_avg = '''
WITH t AS (WITH users AS (
SELECT username, COUNT(rating_id) AS rating_number
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48)

SELECT u.username, COUNT(DISTINCT(rw.review_id)) AS reviews_number
FROM users AS u
LEFT JOIN reviews AS rw ON u.username = rw.username
GROUP BY u.username)

SELECT AVG(reviews_number) AS avg_reviews
FROM t
'''

pd.io.sql.read_sql(sql=text(reviews_avg), con = con)

,avg_reviews
0,24.0


Можно предположить, что написание обзора = прочитанная книга. Значит, активные пользователи, которые ставят много оценок книгам в принципе, в среднем читают 24 книги, что важно при оценке будущих продаж.

**Выводы:**

1. В БД в основном актуальные публикации (80% книг выпущено после 2000 года)
2. БД даёт нам возможность выделить самые популярные издательства для разработки предложений пользователям. Например, самая популярная: Penguin Books
3. Самый популярный по рейтингам автор - это Джоан Роулинг
4. Обзоров на книги достаточно мало (не более 7), но самые популярные мы можем выделить и вынести на первую страницу или отдельный баннер